In [63]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("house-prices/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

import pandas as pd
import numpy as np
import sklearn
# print(dir(sklearn.metrics))
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_squared_error 

X = X_train.copy()
y = y_train.copy()
test = X_test.copy()


# EDA 
pd.set_option('display.max_columns',100)
# display(X.describe(include='all'))
# display(y.describe(include='all'))

# print(X.isna().sum().sort_values(ascending=False)[:20])
# print(test.isnull().sum().sort_values(ascending=False)[:20])
# print(X.info())
# print(y.describe())




# 전처리
X = X[X.describe(exclude='object').columns]
test = test[test.describe(exclude='object').columns]
target = y['SalePrice']
# print(X.info())



# 'id'값 옮겨놓기
X_id = X.pop('Id')
test_id = test.pop('Id')
# print(X.shape)


# 결측치 처리
from sklearn.impute import SimpleImputer
# print(help(sklearn.impute.SimpleImputer))
imp = SimpleImputer()
X = imp.fit_transform(X)
test = imp.fit_transform(test)



# 검증 셋
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X,target, test_size=0.2, random_state=42)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)


# 모델
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr, verbose=False)
pred = xgb.predict(X_val)

from sklearn.metrics import mean_squared_error, mean_squared_log_error

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

print((rmsle(y_val, pred)))

predictions = xgb.predict(test)
output = pd.DataFrame({'Id':test_id, 'SalePrice':predictions})
output.head()
output.shape
rmsle(y_test['SalePrice'], predictions)


(1168, 80) (292, 80) (1168, 2) (292, 2)
(934, 36) (234, 36) (934,) (234,)
36178.25879321676


27367.680255631807

# Insurance_Starter (Tutorial)

In [65]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("insurance/insurance.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='charges')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1070, 7) (268, 7) (1070, 2) (268, 2)


In [126]:
import pandas as pd
import numpy as np
import sklearn
# print(dir(sklearn.metrics))
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_squared_error 

X = X_train.copy()
y = y_train.copy()
test = X_test.copy()


# EDA
# print(X.info())
# print(y.describe())

# print(X.isna().sum())
# print(y.isna().sum())

# 전처리
cols = X.describe(include=['object']).columns
# labelEncoding
# for col in cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     test[col] = le.fit_transform(test[col])
# print(X.info())
# print(cols)
# ohe
X = pd.get_dummies(X, columns=cols)
test = pd.get_dummies(test, columns=cols)
# print(X.head())


# y값 log변환
# print(y.head())
y['charges'] = np.log1p(y['charges'])
# y.describe()


# Standard Scaler
# print(X.describe())
st = StandardScaler()
X['bmi'] = st.fit_transform(X[['bmi']])
test['bmi'] = st.fit_transform(test[['bmi']])


# label encoding
X['age'] = X['age'].apply(lambda x : x//10)
test['age'] = test['age'].apply(lambda x : x//10)
# print(X.head())



# train_validation
target = y['charges']
X = X.drop('id',axis=1)

X_tr, X_val, y_tr, y_val = train_test_split(X, target, test_size=0.2, random_state=42)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)


# rf
rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred.shape)
def rmse2(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
print(rmse2(y_val, pred))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)**2)
print(rmse(y_val, pred))




# ada
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)

print(rmse2(y_val, pred))
print(rmse(y_val, pred))

rf.fit(X, y['charges'])
pred = rf.predict(test.drop('id',axis=1))
pred = np.exp(pred)
output = pd.DataFrame({'id': y_test['id'], 'charges':pred})
output.head()
rmse(y_test['charges'], pred)

(856, 11) (214, 11) (856,) (214,)
0.31119452672944276
0.09684203346636185
0.3939661554924201
0.15520933167347772


26086522.4537485